<a href="https://colab.research.google.com/github/nainye/kaggle_practice/blob/main/notebookc6e702ec37.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## 1. 데이터 준비 및 모듈 임포트

In [ ]:
# 기본 데이터 정리 및 처리
import numpy as np
import pandas as pd

# 시각화
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
plt.style.use('seaborn-whitegrid')
import missingno

# 전처리 및 머신러닝 알고리즘
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import VotingClassifier

# 모델 튜닝 및 평가
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_predict
from sklearn import model_selection

# 경고 제거
import sys
import warnings
warnings.filterwarnings('ignore')

### CSV를 Dataframe으로 변환

In [ ]:
test = pd.read_csv('../input/titanic/test.csv')
train = pd.read_csv('../input/titanic/train.csv')

In [ ]:
# 데이터프레임 확인
train.head()

In [ ]:
train.describe(include='all')

Age, Cabin, Embarked에 결측값이 있음을 확인할 수 있음

In [ ]:
# 각 열의 데이터 종류
train.dtypes

## 2. 파일 병합

In [ ]:
# 병합 준비
ntrain = train.shape[0]
ntest = test.shape[0]

# y_train 결과, test의 승객ID(최종 결과에 넣어야 됨) 따로 빼놓기
y_train = train['Survived'].values
passId = test['PassengerId']

# 병합 파일 만들기
data = pd.concat((train, test))

# 데이터 크기 확인
print("data size is: {}".format(data.shape))

In [ ]:
train['Survived'].value_counts()

train 데이터 인원 중 342명의 생존자, 549명의 사망자

## 3. 파일 탐색

In [ ]:
missingno.matrix(data, figsize=(15, 8))

결측값을 직관적으로 확인 가능

In [ ]:
# 결측값 체크
data.isnull().sum()
# isna()를 써도 똑같음

In [ ]:
# 각 열 이름 확인
data.columns

## 4. 데이터 탐구

In [ ]:
# Co-relation 매트릭스
corr = data.corr()

# 마스크 셋업
mask = np.zeros_like(corr, dtype=np.bool)
mask[np.triu_indices_from(mask)] = True

# 그래프 셋업
plt.figure(figsize=(14, 8))

# 그래프 타이틀
plt.title('Overall Correlation of Titanic Features', fontsize=18)

# Co-relation 매트릭스 런칭
sns.heatmap(corr, mask=mask, annot=False,cmap='RdYlGn', linewidths=0.2, annot_kws={'size':20})

plt.show()

- "Survived" 분석
(0 - Not Survived, 1 - Survived)

In [ ]:
# 생존 여부 0과 1의 숫자를 세어본 후, 그림을 그려보는 countplot 생성
fig = plt.figure(figsize=(10, 2))
sns.countplot(y='Survived', data=train)
print(train.Survived.value_counts())

In [ ]:
def piecount(col):
    f, ax = plt.subplots(1, 2, figsize=(15, 6))
    train[col].value_counts().plot.pie(explode=[0.1 for i in range(train[col].nunique())], autopct='%1.1f%%', ax=ax[0], shadow=True)
    ax[0].set_title(col)
    ax[0].set_ylabel('')
    sns.countplot(col, data=train, ax=ax[1])
    ax[1].set_title(col)
    plt.show()
    
piecount('Survived')  

생존자보다 사망자가 더 많다는 것을 확인할 수 있음

- "Pclass" 분석
(1=1st, 2=2nd, 3=3rd)

In [ ]:
# 각 클래스 당 생존자 확인
pd.crosstab(train.Pclass, train.Survived, margins=True).style.background_gradient(cmap='summer_r')

# train.groupby(['Pclass', 'Survived'])['Survived'].count()

1등급 객실은 생존자가 더 많고, 2등급은 생존자에 사망자가 조금 더 많으나, 3등급은 사망자가 3배 이상 많다는 것을 알 수 있음

In [ ]:
piecount("Pclass")

- "Name" 분석

In [ ]:
data.Name.value_counts()

In [ ]:
# ['Initial'] 열을 새로 만들고, Name에서 추출한 Regular Expression 넣어줌
# 대.소문자 중 .으로 끝나는 부분 추출
temp = data.copy()
temp['Initial'] = 0
temp['Initial'] = data.Name.str.extract('([A-Za-z]+)\.')


In [ ]:
temp['Initial'].value_counts()

In [ ]:
# 생존률 함수
def survpct(col):
    return temp.groupby(col)['Survived'].mean()

survpct('Initial')

In [ ]:
# Dona 결측값을 채우기 위해 나이 확인
temp.loc[temp['Initial'] == 'Dona']

In [ ]:
# 나이로 추측하고 Mrs.로 넣어줌
temp.loc[temp['Initial'] == 'Dona', 'Initial'] = 'Mrs'

In [ ]:
# 성별과 함께 확인
pd.crosstab(temp.Initial, temp.Sex).T.style.background_gradient(cmap='summer_r')

In [ ]:
# Last name은 전부 숫자로 바꿔줌
temp['LastName'] = data.Name.str.extract('([A-Za-z]+)')
temp['NumName'] = temp['LastName'].factorize()[0]
pd.crosstab(temp.NumName, temp.Survived).T.style.background_gradient(cmap='summer_r')

- "Sex" 분석

In [ ]:
def bag(col, target, title, title1):
    f,ax = plt.subplots(1, 2, figsize=(12,5))
    train.groupby([col])[target].mean().plot(kind='bar', ax=ax[0])
    ax[0].set_title(title)
    sns.countplot(col, hue=target, data=train, ax=ax[1])
    ax[1].set_title(title1)
    plt.show()

bag('Sex', 'Survived', 'Survived per Sex', 'Sex Survived vs Not Survived')

탑승객은 남자가 훨씬 더 많지만, 생존 여성 수는 남성 수의 거의 두 배

In [ ]:
pd.crosstab([train.Sex, train.Survived], train.Pclass, margins=True).style.background_gradient(cmap='summer_r')

- "Age" 분석 (빈 값이 많아서 결측값 처리가 중요)

In [ ]:
print('Oldest Passenger was', data['Age'].max(), 'Years')
print('Youngest Passenger was', data['Age'].min(), 'Years')
print('Average Age on the ship was', int(data['Age'].mean()), 'Years')

In [ ]:
f, ax = plt.subplots(1, 2, figsize=(18, 8))
sns.violinplot("Pclass", "Age", hue="Survived", data=train, split=True, ax=ax[0])
ax[0].set_title('Pclass and Age vs Survived')
ax[0].set_yticks(range(0, 110, 10))
sns.violinplot("Sex", "Age", hue="Survived", data=train, split=True, ax=ax[1])
ax[1].set_title('Sex and Age vs Survived')
ax[1].set_yticks(range(0, 110, 10))
plt.show()

### 관찰
1. Pclass에 따라 어린이 수 증가, 어린이의 생존률은 Pclass에 상관없이 양호
2. 1등급 20~50세 승객의 생존 가능성이 높으며, 여성이 더 좋음
3. 남성 생존 확률은 나이 증가에 따라 감소함

177개의 결측값 처리가 필요. but, 평균연령 29세를 4세 아이에게 할당할 수 없기에 Initial별 평균 연령을 보고, Age에 적용

In [ ]:
# Initial별 평균 연령 확인
temp.groupby('Initial').agg({'Age' : ['mean', 'count']})

In [ ]:
# 이니셜 별 평균 연령을 빈 값에 넣어줌
temp = temp.reset_index(drop=True)
temp['Age'] = temp.groupby('Initial')['Age'].apply(lambda x: x.fillna(x.mean()))
temp[31:50]

In [ ]:
# Initial 정리
temp['Initial'].replace(['Capt', 'Col', 'Countess', 'Don', 'Dona' , 'Dr', 'Jonkheer', 'Lady', 'Major', 'Master',  'Miss'  ,'Mlle', 'Mme', 'Mr', 'Mrs', 'Ms', 'Rev', 'Sir'], ['Sacrificed', 'Respected', 'Nobles', 'Mr', 'Mrs', 'Respected', 'Mr', 'Nobles', 'Respected', 'Kids', 'Miss', 'Nobles', 'Nobles', 'Mr', 'Mrs', 'Nobles', 'Sacrificed', 'Nobles'],inplace=True)
temp['Initial'].replace(['Kids', 'Miss', 'Mr', 'Mrs', 'Nobles', 'Respected', 'Sacrificed'], [4, 4, 2, 5, 6, 3, 1], inplace=True)

In [ ]:
temp['Age_Range'] = pd.qcut(temp['Age'], 10) # 10개 값씩 나누어서 범주 생성
survpct('Age_Range')


In [ ]:
temp['Agroup'] = 0
temp.loc[temp['Age'] < 1.0, 'Agroup'] = 1
temp.loc[(temp['Age'] >= 1.0) & (temp['Age'] <= 3.0), 'Agroup'] = 2
temp.loc[(temp['Age'] > 3.0) & (temp['Age'] < 11.0), 'Agroup'] = 7
temp.loc[(temp['Age'] >= 11.0) & (temp['Age'] < 15.0), 'Agroup'] = 13
temp.loc[(temp['Age'] >= 15.0) & (temp['Age'] < 18.0), 'Agroup'] = 16
temp.loc[(temp['Age'] >= 18.0) & (temp['Age'] <=  20.0), 'Agroup'] = 18
temp.loc[(temp['Age'] > 20.0) & (temp['Age'] <= 22.0), 'Agroup'] = 21
temp.loc[(temp['Age'] > 22.0) & (temp['Age'] <= 26.0), 'Agroup'] = 24
temp.loc[(temp['Age'] > 26.0) & (temp['Age'] <= 30.0), 'Agroup'] = 28
temp.loc[(temp['Age'] > 30.0) & (temp['Age'] <= 32.0), 'Agroup'] = 31
temp.loc[(temp['Age'] > 32.0) & (temp['Age'] <= 34.0), 'Agroup'] = 33
temp.loc[(temp['Age'] > 34.0) & (temp['Age'] <= 38.0), 'Agroup'] = 36
temp.loc[(temp['Age'] > 38.0) & (temp['Age'] <= 52.0), 'Agroup'] = 45
temp.loc[(temp['Age'] > 52.0) & (temp['Age'] <= 75.0), 'Agroup'] = 60
temp.loc[temp['Age'] > 75.0, 'Agroup'] = 78

temp.head()

In [ ]:
# 위를 보고 sex를 남, 녀, 1세 이하 Baby로 나눔
temp.loc[(temp['Sex'] == 'male'), 'Sex'] = 1
temp.loc[(temp['Sex'] == 'female'), 'Sex'] = 2
temp.loc[(temp['Age'] < 1), 'Sex'] = 3

survpct('Sex')

- "SibSp"(혼자인지 가족과 함께 있는지) + "Parch"(부모와 함께 탔는지) 분석 ('Alone'그룹과 'Family'그룹으로 나눠보려 함)

In [ ]:
temp.loc[(temp['SibSp'] == 0) & (temp['Parch'] == 0), 'Alone'] = 1
temp['Family'] = temp['Parch'] + temp['SibSp'] + 1

temp.head(n=10)

In [ ]:
bag('Parch', 'Survived', 'Survived per Parch', 'Parch Survived vs Not Survived')

1~3명의 부모와 함께 탑승한 경우의 생존 가능성이 제일 높음

- "Ticket" 분석

- "Fare" 분석

- "Cabin" 분석

- "Embarked" 분석

In [ ]:
# 승선 장소별 생존 확률
sns.factorplot('Embarked', 'Survived', data=temp)
fig = plt.gcf()
fig.set_size_inches(5, 3)
plt.show()

In [ ]:
f,ax = plt.subplots(2, 2, figsize=(20,15))
sns.countplot('Embarked', data=temp, ax=ax[0,0])
ax[0,0].set_title('No. Of Passengers Boarded')
sns.countplot('Embarked', hue='Sex', data=temp, ax=ax[1,0])
ax[0,1].set_title('Male-Female Split for Embarked')
sns.countplot('Embarked', hue='Survived', data=temp, ax=ax[1,0])
ax[1,0].set_title('Embarked vs Survived')
sns.countplot('Embarked', hue='Pclass', data=temp, ax=ax[1,1])
ax[1,1].set_title('Embarked vs Pclass')
plt.subplots_adjust(wspace=0.2, hspace=0.5)

plt.show()

In [ ]:
# 결측값 2개 확인

temp.loc[(temp.Embarked.isnull())]

In [ ]:
# 두 사람의 티켓 번호가 같으므로 같은 티켓 번호를 가진 다른 사람을 찾음
temp.loc[(temp.Ticket == '113572')]

In [ ]:
# 없으므로 가장 비슷한 번호 찾기
temp.sort_values(['Ticket'], ascending=True)[55:70]

In [ ]:
# 앞 뒤로 모두 S이고 Pclass도 모두 1이므로 S일 가능성이 큼
temp.loc[(temp.Embarked.isnull()), 'Embarked'] = 'S'
temp.loc[(temp.Embarked.isnull())]

In [ ]:
temp['Embarked'] = temp['Embarked'].factorize()[0]
temp[11:20]

## 5. Feature Engineering

Priority - Nobles, Women in Pclass 1&2, Babies under 1, Kids under 17 in Pclass 1&2

In [ ]:
temp['Priority'] = 0
temp.loc[(temp['Initial'] == 6), 'Priority'] = 1
temp.loc[(temp['Pclass'] == 1) & (temp['Sex'] == 2), 'Priority'] = 2
temp.loc[(temp['Age'] < 1), 'Priority'] = 3
temp.loc[(temp['Pclass'] == 1) & (temp['Age'] <= 17), 'Priority'] = 4
temp.loc[(temp['Pclass'] == 2) & (temp['Age'] <= 17), 'Priority'] = 5

In [ ]:
survpct('Priority')

In [ ]:
temp.Priority.value_counts()

## 6. 마지막 항목 결정

In [ ]:
temp['F1'] = temp['Priority']
temp['F2'] = temp['Initial']
temp['F3'] = temp['NumName']
temp['F4'] = temp['Family']
temp['F5'] = temp['Embarked']
temp['F6'] = temp['Sex']
temp['F7'] = temp['Pclass']

In [ ]:
temp

In [ ]:
from sklearn.preprocessing import OneHotEncoder, LabelEncoder

# DataFrame 생성
dfl = pd.DataFrame() # for label encoding
good_columns = ['F1', 'F2', 'F3', 'F4', 'F5', 'F6', 'F7']
dfl[good_columns] = temp[good_columns]

dfh = dfl.copy() # for One Hot Encoding

dfl_enc = dfl.apply(LabelEncoder().fit_transform)
dfl_enc.head()

In [ ]:
one_hot_cols = dfh.columns.tolist()
dfh_enc = pd.get_dummies(dfh, columns=one_hot_cols)

dfh_enc.head()

## 7. 머신러닝 모델 만들기

In [ ]:
# 인코딩한 파일을 train과 test로 쪼갬
train = dfh_enc[:ntrain]
test = dfh_enc[ntrain:]

In [ ]:
X_test = test
X_train = train

In [ ]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
ran = RandomForestClassifier(random_state=1)
knn = KNeighborsClassifier()
log = LogisticRegression()
xgb = XGBClassifier()
gbc = GradientBoostingClassifier()
svc = SVC(probability=True)
ext = ExtraTreesClassifier()
ada = AdaBoostClassifier()
gnb = GaussianNB()
gpc = GaussianProcessClassifier()
bag = BaggingClassifier()

# 리스트 준비
models = [ran, knn, log, xgb, gbc, svc, ext, ada, gnb, gpc, bag]
model_names = ['Random Forest', 'K Nearest Neighbor', 'Logistic Regression', 'XGBoost', 'Gradient Boosting', 'SVC', 'Extra Trees', 'AdaBoost', 'Gaussian Naive Bayes', 'Gaussian Process', 'Bagging Classifier']
scores = {}

# 연속적으로 모델을 학습 시키고, 교차 검증
for ind, mod in enumerate(models):
    mod.fit(X_train, y_train)
    acc = cross_val_score(mod, X_train, y_train, scoring="accuracy", cv=10)
    scores[model_names[ind]] = acc

In [ ]:
# 결과 테이블 생성
results = pd.DataFrame(scores).T
results['mean'] = results.mean(1)

result_df = results.sort_values(by='mean', ascending=False)
result_df.head(11)

In [ ]:
result_df = result_df.drop(['mean'], axis=1)
sns.boxplot(data=result_df.T, orient='h')
plt.title('Machine Learning Algorithm Accuracy Score \n')
plt.xlabel('Accuracy Score (%)')

In [ ]:
# 중요도 확인 함수
def importance_plotting(data, xlabel, ylabel, title, n=20):
    sns.set(style="whitegrid")
    ax = data.tail(n).plot(kind='barh')
    
    ax.set(title=title, xlabel=xlabel, ylabel=ylabel)
    ax.xaxis.grid(False)
    ax.yaxis.grid(True)
    plt.show()

In [ ]:
# 데이터 프레임에 항목 중요도 삽입
fi = {'Features' :train.columns.tolist(), 'Importance':xgb.feature_importances_}
importance = pd.DataFrame(fi, index=fi['Features']).sort_values('Importance', ascending=True)

In [ ]:
# 그래프 제목
title = 'Top 20 most important features in predicting survival on the Titanic: XGB'

# 그래프 그리기
importance_plotting(importance, 'Importance', 'Features', title, 20)

In [ ]:
# 중요도를 데이터 프레임에 넣음. Logistic regression에서는 중요도보다 coefficients를 사용

# Features라는 열에 train의 열들의 이름을 리스트로 만들어 넣고,
# Importance에는 Logistic regression의 coefficient를 바꾸어 넣으라는 넘파이 명령. (즉, 가로를 세로로)
fi = {'Features':train.columns.tolist(), 'Importance':np.transpose(log.coef_[0])}
importance = pd.DataFrame(fi, index=fi['Features']).sort_values('Importance', ascending=True)

# 그래프 제목
title = 'Top 20 important features in predicting survival on the Titanic: Logistic Regression'

# 그래프 그리기
importance_plotting(importance, 'Importance', 'Features', title, 20)

In [ ]:
# 5가지 모델에 대한 항목 중요도 얻기
gbc_imp = pd.DataFrame({'Feature':train.columns, 'gbc importance':gbc.feature_importances_})
xgb_imp = pd.DataFrame({'Feature':train.columns, 'xgb importance':xgb.feature_importances_})
ran_imp = pd.DataFrame({'Feature':train.columns, 'ran importance':ran.feature_importances_})
ext_imp = pd.DataFrame({'Feature':train.columns, 'ext importance':ext.feature_importances_})
ada_imp = pd.DataFrame({'Feature':train.columns, 'ada importance':ada.feature_importances_})

# 이를 하나의 데이터 프레임으로
importances = gbc_imp.merge(xgb_imp, on='Feature').merge(ran_imp, on='Feature').merge(ext_imp, on='Feature').merge(ada_imp, on='Feature')

# 항목 당 평균 중요도
importances['Average'] = importances.mean(axis=1)

#랭킹 정하기
importances = importances.sort_values(by='Average', ascending=False).reset_index(drop=True)

## 8. 중요도에 따라 모델 재설정

In [ ]:
# 중요도를 다시 데이터프레임에 넣음
fi = {'Features':importances['Feature'], 'Importance':importances['Average']}
importance = pd.DataFrame(fi).set_index('Features').sort_values('Importance', ascending=True)

# 그래프 제목
title = 'Top 20 important features in predicting survival on the Titanic: 5 model average'

# 그래프 그리기
importance_plotting(importance, 'Importance', 'Features', title, 20)

In [ ]:
importance1 = importance[-381:]

importance1[371:381]

In [ ]:
# 영양가 있는 380개만 넣음
mylist = list(importance1.index)

In [ ]:
train1 = pd.DataFrame()
test1 = pd.DataFrame()

for i in mylist:
    train1[i] = train[i]
    test1[i] = test[i]
    
train1.head()

In [ ]:
train = train1
test = test1

# 모델의 변수 다시 정의
X_train = train
X_test = test

# 바꿔줌
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
ran = RandomForestClassifier(random_state=1)
knn = KNeighborsClassifier()
log = LogisticRegression()
xgb = XGBClassifier(random_state=1)
gbc = GradientBoostingClassifier(random_state=1)
svc = SVC(probability=True)
ext = ExtraTreesClassifier(random_state=1)
ada = AdaBoostClassifier(random_state=1)
gnb = GaussianNB()
gpc = GaussianProcessClassifier()
bag = BaggingClassifier(random_state=1)

# 리스트 준비
models = [ran, knn, log, xgb, gbc, svc, ext, ada, gnb, gpc, bag]         
model_names = ['Random Forest', 'K Nearest Neighbor', 'Logistic Regression', 'XGBoost', 'Gradient Boosting', 'SVC', 'Extra Trees', 'AdaBoost', 'Gaussian Naive Bayes', 'Gaussian Process', 'Bagging Classifier']
scores2 = {}

# 학습 및 교차 검증
for ind, mod in enumerate(models):
    mod.fit(X_train, y_train)
    acc = cross_val_score(mod, X_train, y_train, scoring = "accuracy", cv = 10)
    scores2[model_names[ind]] = acc

In [ ]:
# 결과 테이블 생성
results = pd.DataFrame(scores2).T
results['mean'] = results.mean(1)

result_df = results.sort_values(by='mean', ascending=False)
result_df.head(11)

In [ ]:
result_df = result_df.drop(['mean'], axis=1)
sns.boxplot(data=result_df.T, orient='h')
plt.title('Machine Learning Algorithm Acuuracy Score \n')
plt.xlabel('Accuracy Score (%)')

## 9. 하이퍼 파라미터 튜닝

SVC

In [ ]:
# 파라미터 서치
Cs = [0.01, 0.1, 1, 5, 10, 15, 20, 50]
gammas = [0.001, 0.01, 0.1]

# 파라미터 그리드 셋팅
hyperparams = {'C': Cs, 'gamma': gammas}

# 교차검증
gd = GridSearchCV(estimator=SVC(probability=True), param_grid=hyperparams, verbose=True, cv=5, scoring="accuracy", n_jobs=-1)

# 모델 fitting
gd.fit(X_train, y_train)

# 결과
print(gd.best_score_)
print(gd.best_params_)

Gradient Boosting Classifier

In [ ]:
learning_rate = [0.01, 0.05, 0.1, 0.2, 0.5]
n_estimators = [100, 1000, 2000]
max_depth = [3, 5, 10, 15]

hyperparams = {'learning_rate': learning_rate, 'n_estimators': n_estimators}

gd = GridSearchCV(estimator=GradientBoostingClassifier(), param_grid = hyperparams, verbose=True, cv=5, scoring="accuracy", n_jobs=-1)

gd.fit(X_train, y_train)
print(gd.best_score_)
print(gd.best_params_)

Logistic Regression

In [ ]:
penalty = ['l1', 'l2']
C = np.logspace(0, 4, 10)

hyperparams = {'penalty': penalty, 'C': C}

gd = GridSearchCV(estimator=LogisticRegression(), param_grid=hyperparams, verbose=True, cv=5, scoring="accuracy", n_jobs=-1)

gd.fit(X_train, y_train)
print(gd.best_score_)
print(gd.best_params_)

XGBoost Step 1.

In [ ]:
learning_rate = [0.001, 0.005, 0.01, 0.05, 0.1, 0.2]
n_estimators = [10, 50, 100, 250, 500, 1000]

hyperparams = {'learning_rate': learning_rate, 'n_estimators': n_estimators}

gd = GridSearchCV(estimator = XGBClassifier(), param_grid=hyperparams, verbose=True, cv=5, scoring="accuracy", n_jobs=-1)

gd.fit(X_train, y_train)
print(gd.best_score_)
print(gd.best_params_)

XGB Step 2.

In [ ]:
max_depth = [3, 4, 5, 6, 7, 8, 9, 10]
min_child_weight = [1, 2, 3, 4, 5, 6]

hyperparams = {'max_depth': max_depth, 'min_child_weight': min_child_weight}

gd = GridSearchCV(estimator=XGBClassifier(learning_rate=0.2, n_estimators=50),
                  param_grid=hyperparams, verbose=True, cv=5, scoring="accuracy", n_jobs=-1)

gd.fit(X_train, y_train)
print(gd.best_score_)
print(gd.best_params_)

XGB Step 3.

In [ ]:
gamma = [i*0.1 for i in range(0,5)]

hyperparams = {'gamma': gamma}

gd = GridSearchCV(estimator= XGBClassifier(learning_rate=0.2, n_estimators=50, max_depth=8, min_child_weight=2), 
                  param_grid=hyperparams, verbose=True, cv=5, scoring="accuracy", n_jobs=-1)

gd.fit(X_train, y_train)
print(gd.best_score_)
print(gd.best_params_)

XGB Step 4.

In [ ]:
subsample = [0.6, 0.65, 0.7, 0.75, 0.8, 0.85, 0.9, 0.95, 1]
colsample_bytree = [0.6, 0.65, 0.7, 0.75, 0.8, 0.85, 0.9, 0.95, 1]

hyperparams = {'subsample': subsample, 'colsample_bytree': colsample_bytree}

gd = GridSearchCV(estimator=XGBClassifier(learning_rate=0.2, n_estimators=50, max_depth=8, min_child_weight=2, gamma=0), 
                  param_grid=hyperparams, verbose=True, cv=5, scoring="accuracy", n_jobs=-1)

gd.fit(X_train, y_train)

print(gd.best_score_)
print(gd.best_params_)

XGB Step 5.

In [ ]:
reg_alpha = [1e-5, 1e-2, 0.1, 1, 100]

hyperparams = {'reg_alpha': reg_alpha}

gd = GridSearchCV(estimator=XGBClassifier(learning_rate=0.2, n_estimator=50, max_depth=8, min_child_weight=2, gamma=0, subsample=0.7, colsample_bytree=0.65),
                 param_grid=hyperparams, verbose=True, cv=5, scoring="accuracy", n_jobs=-1)

gd.fit(X_train, y_train)
print(gd.best_score_)
print(gd.best_params_)

Gaussian Process

In [ ]:
n_restarts_optimizer = [0, 1, 2, 3]
max_iter_predict = [1, 2, 5, 10, 20, 35, 50, 100]
warm_start = [True, False]

hyperparams = {'n_restarts_optimizer': n_restarts_optimizer, 'max_iter_predict': max_iter_predict, 'warm_start': warm_start}

gd = GridSearchCV(estimator=GaussianProcessClassifier(), param_grid=hyperparams, verbose=True, cv=5, scoring="accuracy", n_jobs=-1)

gd.fit(X_train, y_train)
print(gd.best_score_)
print(gd.best_params_)

Adaptive Boost

In [ ]:
n_estimators = [10, 100, 200, 500]
learning_rate = [0.001, 0.01, 0.1, 0.5, 1, 1.5, 2]

hyperparams = {'n_estimators': n_estimators, 'learning_rate': learning_rate}

gd = GridSearchCV(estimator=AdaBoostClassifier(), param_grid=hyperparams, verbose=True, cv=5, scoring="accuracy", n_jobs=-1)

gd.fit(X_train, y_train)
print(gd.best_score_)
print(gd.best_params_)

KNN

In [ ]:
n_neighbors = [1, 2, 3, 4, 5]
algorithm = ['auto']
weights = ['uniform', 'distance']
leaf_size = [1, 2, 3, 4, 5, 10]

hyperparams = {'algorithm':algorithm, 'weights': weights, 'leaf_size': leaf_size, 'n_neighbors': n_neighbors}

gd=GridSearchCV(estimator=KNeighborsClassifier(), param_grid=hyperparams, verbose=True, cv=5, scoring="accuracy", n_jobs=-1)

gd.fit(X_train, y_train)
print(gd.best_score_)
print(gd.best_params_)

Random Forest

In [ ]:
n_estimators = [10, 50, 100, 200]
max_depth = [3, None]
max_features = [0.1, 0.2, 0.5, 0.8]
min_samples_split = [2, 6]
min_samples_leaf = [2, 6]

hyperparams = {'n_estimators': n_estimators, 'max_depth': max_depth, 'max_features': max_features,
              'min_samples_split': min_samples_split, 'min_samples_leaf': min_samples_leaf}

gd = GridSearchCV(estimator=RandomForestClassifier(), param_grid=hyperparams, verbose=True, cv=5, scoring="accuracy", n_jobs=-1)

gd.fit(X_train, y_train)
print(gd.best_score_)
print(gd.best_params_)

Extra Trees

In [ ]:
n_estimators = [10, 25, 50, 75, 100]
max_depth = [3, None]
max_features = [0.1, 0.2, 0.5, 0.8]
min_samples_split = [2, 10]
min_samples_leaf = [2, 10]

hyperparams = {'n_estimators': n_estimators, 'max_depth': max_depth, 'max_features': max_features, 'min_samples_split': min_samples_split, 'min_samples_leaf': min_samples_leaf}

gd = GridSearchCV(estimator=ExtraTreesClassifier(), param_grid=hyperparams, verbose=True, cv=5, scoring="accuracy", n_jobs=-1)

gd.fit(X_train, y_train)
print(gd.best_score_)
print(gd.best_params_)

Bagging Classifier

In [ ]:
n_estimators = [10, 50, 75, 100, 200]
max_samples = [0.1, 0.2, 0.5, 0.8, 1.0]
max_features = [0.1, 0.2, 0.5, 0.8, 1.0]

hyperparams = {'n_estimators': n_estimators, 'max_samples': max_samples, 'max_features': max_features}

gd = GridSearchCV(estimator=BaggingClassifier(), param_grid=hyperparams, verbose=True, cv=5, scoring="accuracy", n_jobs=-1)

gd.fit(X_train, y_train)
print(gd.best_score_)
print(gd.best_params_)

## 10. 모델 재 트레이닝

In [ ]:
# 튜닝 모델 시작
# sample을 split하는 것은 전체 데이터의 80%를 train 셋에 20%는 test 셋에 줌

ran = RandomForestClassifier(max_depth=None, max_features=0.1, min_samples_leaf=2, min_samples_split=6, n_estimators=100, random_state=1)

knn = KNeighborsClassifier(leaf_size=1, n_neighbors=5, weights='distance')

log = LogisticRegression(C=1.0, penalty='l2')

xgb = XGBClassifier(learning_rate=0.2, n_estimators=50, max_depth=8, min_child_weight=2, gamma=0, subsample=0.7, colsample_bytree=0.65, reg_alpha=1)

gbc = GradientBoostingClassifier(learning_rate=0.2, max_depth=3, n_estimators=1000)

svc = SVC(probability=True, gamma=0.001, C=20)

ext = ExtraTreesClassifier(max_depth=None, max_features=0.1, min_samples_leaf=2, min_samples_split=2, n_estimators=50, random_state=1)

ada = AdaBoostClassifier(learning_rate=0.5, n_estimators=200, random_state=1)

gpc = GaussianProcessClassifier(max_iter_predict=1, n_restarts_optimizer=0, warm_start=True)

bag = BaggingClassifier(max_features=0.5, max_samples=1.0, n_estimators=75, random_state=1)

# 리스트
models = [ran, knn, log, xgb, gbc, svc, ext, ada, gnb, gpc, bag]         
model_names = ['Random Forest', 'K Nearest Neighbour', 'Logistic Regression', 'XGBoost', 'Gradient Boosting', 'SVC', 'Extra Trees', 'AdaBoost', 'Gaussian Naive Bayes', 'Gaussian Process', 'Bagging Classifier']
scores3 = {}

# Sequentially fit and cross validate all models
for ind, mod in enumerate(models):
    mod.fit(X_train, y_train)
    acc = cross_val_score(mod, X_train, y_train, scoring = "accuracy", cv = 10)
    scores3[model_names[ind]] = acc

In [ ]:
results = pd.DataFrame(scores).T
results['mean'] = results.mean(1)
result_df = results.sort_values(by='mean', ascending=False)
result_df.head(11)

result_df = result_df.drop(['mean'], axis=1)
sns.boxplot(data=result_df.T, orient='h')
plt.title('Machine Learning Algorithm Accuracy Score \n')
plt.xlabel('Accuracy Score (%)')

## 11. 마지막 보팅

In [ ]:
# 튜닝한 파라미터로 하드보팅
grid_hard = VotingClassifier(estimators = [('Random Forest', ran),
                                           ('Logistic Regression', log),
                                           ('XGBoost', xgb),
                                           ('Gradient Boosting', gbc),
                                           ('Extra Trees', ext),
                                           ('AdaBoost', ada),
                                           ('Gaussian Process', gpc),
                                           ('SVC', svc),
                                           ('K Nearest Neighbor', knn),
                                           ('Bagging Classifier', bag)], voting='hard')

grid_hard_cv = model_selection.cross_validate(grid_hard, X_train, y_train, cv=10)
grid_hard.fit(X_train, y_train)

print("Hard voting on test set score mean: {:.2f}".format(grid_hard_cv['test_score'].mean()*100))

In [ ]:
# 튜닝한 파라미터로 소프트보팅
grid_soft = VotingClassifier(estimators = [('Random Forest', ran),
                                           ('Logistic Regression', log),
                                           ('XGBoost', xgb),
                                           ('Gradient Boosting', gbc),
                                           ('Extra Trees', ext),
                                           ('AdaBoost', ada),
                                           ('Gaussian Process', gpc),
                                           ('SVC', svc),
                                           ('K Nearest Neighbor', knn),
                                           ('Bagging Classifier', bag)], voting='soft')

grid_soft_cv = model_selection.cross_validate(grid_soft, X_train, y_train, cv=10)
grid_soft.fit(X_train, y_train)

print("Soft voting on test set score mean: {:.2f}".format(grid_soft_cv['test_score'].mean()*100))

## 12. 마지막 모델 예측

In [ ]:
# 마지막 예측 hard
predictions = grid_hard.predict(X_test)

submission = pd.concat([pd.DataFrame(passId), pd.DataFrame(predictions)], axis='columns')

submission.columns = ["PassengerId", "Survived"]
submission.to_csv('titanic_submission1.csv', header=True, index=False)

In [ ]:
# 마지막 예측 soft
predictions = grid_soft.predict(X_test)

submission = pd.concat([pd.DataFrame(passId), pd.DataFrame(predictions)], axis='columns')

submission.columns = ["PassengerId", "Survived"]
submission.to_csv('titanic_submission2.csv', header=True, index=False)

hard voting과 soft voting 모두를 최종 제출에 대한 최종 예측을 하는데 사용

그 결과로 csv 파일 생성